In [1]:
import json
import pandas as pd
import matplotlib as plt

%matplotlib inline

def gen_fjson(filename):
    """
    Iterate over objects in an FJSON file.
    """
    with open(filename) as f:
        for line in f:
            try:
                yield json.loads(line)
            except:
                pass

def rejoin_tfo_df(tfo_rdf):
    tfo_df = tfo_rdf.loc[:,['tfostate','connstate','dip','host','rank',
                            'fwd_rst','rev_rst','tfo_seq','tfo_ack','tfo_dlen','tfo_synclen']]

    tfo_0df = tfo_df[tfo_df['tfostate'] == 0].copy()
    tfo_0df.index = tfo_0df.dip
    del(tfo_0df['tfostate'])
    del(tfo_0df['dip'])
    del(tfo_0df['tfo_seq'])
    del(tfo_0df['tfo_ack'])
    del(tfo_0df['tfo_dlen'])
    del(tfo_0df['tfo_synclen'])
    tfo_0df.columns = ['conn_t0','host','rank','fwd_rst_t0','rev_rst_t0']

    tfo_1df = tfo_df[tfo_df['tfostate'] == 1].copy()
    tfo_1df.index = tfo_1df.dip
    del(tfo_1df['tfostate'])
    del(tfo_1df['dip'])
    del(tfo_1df['host'])
    del(tfo_1df['rank'])
    tfo_1df.columns = ['conn_t1','fwd_rst_t1','rev_rst_t1','tfo_seq','tfo_ack','tfo_dlen','tfo_synclen']
    tfo_1df['tfo_dao'] = tfo_1df['tfo_ack'] - tfo_1df['tfo_seq'] - tfo_1df['tfo_dlen'] - 1
    
    tfo_jdf = tfo_0df.join(tfo_1df, how="inner")
    tfo_xdf = tfo_0df.loc[tfo_0df.index.difference(tfo_1df.index)]
    
    return(tfo_jdf, tfo_xdf)

In [2]:
VPS = ('ams3','lon1','fra1','sgp1','tor1','sfo2')

## path-dependent cookie negotiation? 
not really, just transients.

In [3]:
tfo_cookie_jdfs = []
tfo_cookie_xdfs = []

tfo_cookie_jdf = []

for run in range(0,3):
    tfo_cookie_jdfs.append({})
    tfo_cookie_xdfs.append({})
    
    for vp in VPS:
        (tfo_cookie_jdfs[run][vp], tfo_cookie_xdfs[run][vp]) = \
            rejoin_tfo_df(pd.DataFrame(gen_fjson("cookie6_run"+str(run)+"_vp"+vp+".fjson")))
        tfo_cookie_jdfs[run][vp] = tfo_cookie_jdfs[run][vp].loc[:,['tfo_synclen','tfo_dao']]
        tfo_cookie_jdfs[run][vp].columns = (str(run)+"_"+vp+"_cookie", str(run)+"_"+vp+"_dao")

    tfo_cookie_jdf.append(tfo_cookie_jdfs[run][VPS[0]])
    for vp in VPS[1:]:
        tfo_cookie_jdf[run] = tfo_cookie_jdf[run].join(tfo_cookie_jdfs[run][vp], how="inner")

tfo_cookie_df = tfo_cookie_jdf[0]
for run in range(1,3):
    tfo_cookie_df = tfo_cookie_df.join(tfo_cookie_jdf[run], how="inner")

In [4]:
cookie_cols = []
for run in range(0,3):
    for vp in VPS:
        cookie_cols.append("_".join((str(run), vp, "cookie")))

all_cookie_df = tfo_cookie_df[tfo_cookie_df.loc[:,cookie_cols].sum(axis=1) == 8 * len(cookie_cols)]
not_all_cookie_df = tfo_cookie_df.loc[tfo_cookie_df.index.difference(all_cookie_df.index)]

In [5]:
len(not_all_cookie_df)

16

## stable TFO-dependent connectivity?

In [6]:
tfo_cfail_jdfs = {}
tfo_cfail_xdfs = {}

for vp in VPS:
    (tfo_cfail_jdfs[vp], tfo_cfail_xdfs[vp]) = \
        rejoin_tfo_df(pd.DataFrame(gen_fjson("cfail6_run0_vp"+vp+".fjson")))
    tfo_cfail_jdfs[vp] = tfo_cfail_jdfs[vp].loc[:,['conn_t0','conn_t1']]
    tfo_cfail_jdfs[vp].columns = (vp+"_c0", vp+"_c1")
    
tfo_cfail_jdf = tfo_cfail_jdfs[VPS[0]]
for vp in VPS[1:]:
    tfo_cfail_jdf = tfo_cfail_jdf.join(tfo_cfail_jdfs[vp], how="inner")

In [7]:
tfo_cfail_jdf['c0'] = tfo_cfail_jdf.loc[:,[vp+"_c0" for vp in VPS]].sum(axis=1)
tfo_cfail_jdf['c1'] = tfo_cfail_jdf.loc[:,[vp+"_c1" for vp in VPS]].sum(axis=1)

In [8]:
tfo_cfail_jdf[(tfo_cfail_jdf['c0'] == len(VPS)) & (tfo_cfail_jdf['c1'] == 0)]

,ams3_c0,ams3_c1,lon1_c0,lon1_c1,fra1_c0,fra1_c1,sgp1_c0,sgp1_c1,tor1_c0,tor1_c1,sfo2_c0,sfo2_c1,c0,c1
dip,,,,,,,,,,,,,,
101.251.112.199,True,False,True,False,True,False,True,False,True,False,True,False,6,0
222.73.86.116,True,False,True,False,True,False,True,False,True,False,True,False,6,0
222.186.191.198,True,False,True,False,True,False,True,False,True,False,True,False,6,0
61.67.176.168,True,False,True,False,True,False,True,False,True,False,True,False,6,0
